In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import csv
import pickle
from ytmusicapi import YTMusic
ytmusic = YTMusic("headers_auth.json")



In [ ]:
def get_all_playlist_songs(playlist_id):
    point = 0
    songs = []
    songstrings = []
    while True:
        songs_list = sp.playlist_tracks(playlist_id,limit = 20, offset=point)['items']
        if len(songs_list) == 0:
            break
        else:
            songs = songs + songs_list
            point = point + 20
    for idx, track in enumerate(songs):
        #print(point, "   |||   Artist: ", track['track']['artists'][0]['name'], "  |||   Title: ",track['track']['name'])
        artist = track['track']['artists'][0]['name']
        name = track['track']['name']
        full_search = name + ' ' + artist
        songstrings.append(full_search)
    return songstrings

def make_yt_playlist(playlistname):
    playlistId = ytmusic.create_playlist(playlistname, "description")
    return playlistId

def search_youtube_for_song(songstring):
    search_results = []
    search_results = ytmusic.search(songstring)
    ii = 0
    for x in search_results:
        ii =ii +1
        try:
            if ii > 10:
                print("Failed: ", songstring)
                videoId = ""
                break
                
            else:
                videoId = x['videoId']
                break
                
        except:
            print("not first song")
            continue   
    return videoId

def add_song_to_YT_playlist(playlistId, videoID):
    ytmusic.add_playlist_items(playlistId, [videoID])
    

In [ ]:
scope = "user-library-read"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
playlist_list = []
playlist_ids = []
offset_i = 0
for ik in range(10):
    playlist = sp.current_user_playlists(offset= offset_i, limit= 20)
    offset_i = offset_i + 20
    for i in playlist['items']:
        print(i['name'])
        playlist_list.append(i['name'])
        playlist_ids.append(i['id'])

In [ ]:
len(playlist_list)

In [ ]:
for idx, plname in enumerate(playlist_list):
    playlistID = make_yt_playlist(plname)
    songsearch = get_all_playlist_songs(playlist_ids[idx])
    for i in songsearch:
        vidID = search_youtube_for_song(i)
        if len(vidID) != 0:
            add_song_to_YT_playlist(playlistID,vidID)